In [1]:
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error

sys.path.append("../")

from old_code.RGS import FastRandomizedGreedySelection, RandomizedGreedySelection

# Generate synthetic data
np.random.seed(42)
n, p = 5000, 500  # 5000 samples, 500 features

# Create base features
X = np.random.randn(n, p)

X[:, 1] = 0 * X[:, 0] + 0.3 * np.random.randn(n)  # Correlated with feature 0
X[:, 3] = 0 * X[:, 2] + 0.4 * np.random.randn(n)  # Correlated with feature 2
X[:, 5] = 0 * X[:, 0] + 0 * X[:, 4] + 0.1 * np.random.randn(n)  # Correlated with features 0 and 4
true_coef = np.zeros(p)
true_coef[:6] = [1.5, -0.8, 1.2, -0.5, 0.7, -0.3]  # Only first 6 features are relevant
y = X @ true_coef + np.random.normal(0, 1, n)  # Add some noise

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Compare running time

In [2]:
rgs = RandomizedGreedySelection(k=10, m=200) # Original implementation
rgs.fit(X_train, y_train) # 27.9s

In [3]:
rgs1 = FastRandomizedGreedySelection(k=10, m=200)
rgs1.fit(X_train, y_train) # 7.9s

In [4]:
# Keep only feature sets appearing more than 1 time, we increase the number of estimators to 5000
rgs2 = FastRandomizedGreedySelection(k=10, m=200, tol=1, n_estimators=5000) 
rgs2.fit(X_train, y_train) # 7.7s

The problem with the above approach is that at each step, the total number of feature sets goes down. We also introduce a small amount of bias. We can overcome this using resampling.

In [5]:
rgs3 = FastRandomizedGreedySelection(k=10, m=200, tol=0, resample=True)
rgs3.fit(X_train, y_train) # 5.9s

In [6]:
# Both resample and filter. No. of estimators increased to 2500
rgs4 = FastRandomizedGreedySelection(k=10, m=200, tol=1, n_estimators=2500, resample=True)
rgs4.fit(X_train, y_train) # 6.4s

In [7]:
lasso = LassoCV()
lasso.fit(X_train, y_train) # 0.4s

LassoCV()

# Compare prediction accuracy

In [8]:
estimators = {"rgs" : rgs,
              "rgs1" : rgs1,
              "rgs2" : rgs2,
              "rgs3" : rgs3,
              "rgs4" : rgs4,
              "lasso" : lasso
              }
results = {}
for name, estimator in estimators.items():
    results[name] = mean_squared_error(estimator.predict(X_test), X_test @ true_coef)
results

{'rgs': 0.00605774981588647,
 'rgs1': 0.006397642366785582,
 'rgs2': 0.007304560456249431,
 'rgs3': 0.005993455427545296,
 'rgs4': 0.006570436937307765,
 'lasso': 0.0148757421939166}

Seems like the resampling trick seems to reduce running time by a bit but without sacrificing accuracy.

# Compare feature ranking

Note that in our generative model, the relevant features are [0, 1, 2, 3, 4, 5]

In [9]:
np.abs(lasso.coef_).argsort()[::-1] # Feature 5 only shows up in ~170th place!

array([  0,   2,   4,   1,   3, 439, 321, 224, 230, 494, 428, 129, 434,
       303,  57, 339,  90, 206, 176, 471, 343, 462, 123, 480, 263, 147,
       112, 484,  25, 311, 292,  49, 497, 223,   9, 460, 402, 183, 131,
       209, 418, 394,  80, 210, 499, 180, 141, 195, 143, 493, 461, 477,
       163, 152, 166, 144, 145, 146, 479, 148, 149, 150, 151, 153, 162,
       154, 165, 156, 164, 157, 158, 159, 160, 161, 155, 454, 142, 140,
       115, 116, 117, 118, 119, 120, 121, 122, 483, 124, 125, 126, 127,
       128, 482, 130, 481, 132, 133, 134, 135, 136, 137, 138, 139, 167,
       170, 168, 212, 201, 202, 203, 204, 205, 476, 207, 208, 475, 474,
       211, 213, 199, 214, 215, 216, 217, 218, 219, 220, 221, 222, 473,
       472, 200, 198, 169, 182, 113, 171, 172, 173, 174, 175, 478, 177,
       178, 179, 181, 184, 197, 185, 186, 187, 188, 189, 190, 191, 192,
       193, 194, 196, 114, 485, 226, 111,  30,  31,  32,  33,  34,  35,
        36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  4

In [10]:
np.abs(rgs1.coef_).argsort()[::-1] # Feature 5 shows up in 6th place

array([  0,   2,   1,   4,   3,   5, 439, 321, 224, 230, 428, 129, 494,
       434, 105,  25, 209, 343, 339,  60, 303, 176, 112, 480,  57, 210,
       123,  83, 449,  32, 402, 471, 143, 393, 409, 206, 386,  90, 472,
       311, 462, 270, 195, 427, 380, 170, 436, 141, 499, 497, 285, 280,
       310, 484,  80, 479, 418,  53, 131, 153, 265, 347, 263, 104, 451,
       275, 271,  41, 160, 490, 154, 151, 152, 150, 119, 122, 157, 155,
       156, 148, 158, 159, 118, 161, 162, 163, 149, 145, 147, 135, 124,
       125, 126, 127, 128, 482, 130, 132, 133, 134, 136, 146, 121, 137,
       138, 139, 140, 481, 142, 120, 144, 483, 164, 169, 165, 205, 194,
       476, 196, 197, 198, 199, 200, 201, 202, 203, 204, 475, 166, 207,
       208, 474, 473, 211, 212, 213, 214, 215, 216, 217, 193, 192, 191,
       190, 167, 168, 116, 478, 171, 172, 173, 174, 175, 477, 177, 178,
       179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 117, 450,
       115,  43,  30,  31,  33,  34,  35,  36,  37,  38,  39,  4

# Compare total number of feature subsets at step k

In [11]:
n_subsets = {}
for name, estimator in estimators.items():
    if name not in ["lasso", "rgs"]:
        n_subsets[name] = [sum(estimator.trajectory[j].values()) for j in range(10)]
pd.DataFrame(n_subsets)

,rgs1,rgs2,rgs3,rgs4
0,1000,5000,1000,2500
1,1000,5000,1000,2500
2,1000,4995,1000,2499
3,1000,4986,1000,2491
4,1000,4915,1000,2475
5,1000,4771,1000,2431
6,1000,4509,1000,2406
7,1000,4199,1000,2356
8,1000,3763,1000,2335
9,1000,3348,1000,2315
